In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## Parameters and Global variables
district = "all"


## Load and Initialize Data

In [5]:
## Population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Number of regions
r = len(initDataDF.index)

## T-Matrix
tData = pd.read_csv("../output/" + district + "_tmatrix_results.csv")

## Sort distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)

In [6]:
tData.head(5)
tData[tData.isnull().any(axis=1)]
# TODO fix the unnamed colum

,Unnamed: 0,iName,jName,Tij


## Prepare T-Matrix

In [7]:

tMatrix = tData['Tij'].to_numpy().reshape(r,r)
tMatrix

array([[6.67758000e+03, 2.04013815e-01, 1.27738582e-02, ...,
        1.71236727e-01, 3.10475222e-02, 7.98414545e-03],
       [1.56496093e-01, 2.90954300e+04, 7.45440709e-02, ...,
        1.53585706e-01, 5.24088576e-02, 4.88968441e-02],
       [9.12982284e-03, 6.94559477e-02, 3.06105800e+04, ...,
        1.20976746e-02, 1.43104373e-02, 4.47375863e+00],
       ...,
       [2.26326396e-01, 2.64633709e-01, 2.23717485e-02, ...,
        9.82944000e+03, 5.35883146e-02, 1.38572077e-02],
       [5.32014866e-02, 1.17073228e-01, 3.43091112e-02, ...,
        6.94750121e-02, 2.81433600e+04, 2.02950345e-02],
       [6.30217340e-03, 5.03151550e-02, 4.94076502e+00, ...,
        8.27559724e-03, 9.34877871e-03, 1.65401600e+04]])

In [8]:
tColSum = tMatrix.sum(axis=0)
tColSum
#tMatrix[0]/initDataDF['N']
#tMatrix[1]/tColSum[0]
#np.multiply(tMatrix[0]/Ni,tMatrix[1]/tColSum).sum()

array([ 7116.24111591, 30962.62130169, 33086.53479988, ...,
       10060.05380915, 29005.93804394, 17897.24073407])

## Function Definition

In [9]:
def getCijv3(i, j, tMatrix, Ni):
    cwSum = np.multiply(tMatrix[i]/Ni,tMatrix[j]/tColSum).sum()
    return cwSum

In [10]:
def getCijv2(i, j, tMatrix, Ni):
    cwSum = 0
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        Tjk = tMatrix[j][k]
        TlkSum = tMatrix.sum(axis=0)[k]
        
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)

    return cwSum

In [11]:
def getCij(i, j, tMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    cwSum = 0
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        Tjk = tMatrix[j][k]
        TlkSum = 0;

        for l in range(0, r-1):
            TlkSum = TlkSum + tMatrix[l][k]
        
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)

    return cwSum

In [12]:
getCij(0,1, tMatrix, initDataDF)

5.31551802829101e-05

## Compute C-Matrix

In [13]:
data = []
for i in range(0,r):
    Ni = initDataDF.iloc[i]['N']
    for j in range(0,r):
        Cij = getCijv3(i,j,tMatrix, Ni)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Cij})

resultsDF = pd.DataFrame(data);


## Save Output

In [14]:
resultsDF.to_csv("../output/" + district + "_cmatrix_results.csv")

In [15]:
resultsDF.head(10)

,iName,jName,Cij
0,ARNagar__Malappuram,ARNagar__Malappuram,0.854012
1,ARNagar__Malappuram,Adat__Thrissur,0.000053
2,ARNagar__Malappuram,Adichanalloor__Kollam,0.000003
3,ARNagar__Malappuram,Adimali__Idukki,0.000012
4,ARNagar__Malappuram,Adoor(M)__Pathanamthitta,0.000006
5,ARNagar__Malappuram,Agali__Palakkad,0.000046
6,ARNagar__Malappuram,Aiykkaranadu__Ernakulam,0.000007
7,ARNagar__Malappuram,Ajanur__Kasaragod,0.000014
8,ARNagar__Malappuram,Akalakunnam__Kottayam,0.000004
9,ARNagar__Malappuram,Akathethara__Palakkad,0.000038
